In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import gc
import sys
import warnings
warnings.filterwarnings("ignore")

In [2]:
# uplod kaggle.json
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"saeidehnajafzadeh","key":"6288d0f09353bef35d879717fb5a1834"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c bosch-production-line-performance


 94% 638M/680M [00:06<00:01, 41.4MB/s]
100% 680M/680M [00:06<00:00, 103MB/s] 


In [7]:
!unzip bosch-production-line-performance.zip -d bosch_data


Archive:  bosch-production-line-performance.zip
  inflating: bosch_data/sample_submission.csv.zip  
  inflating: bosch_data/test_categorical.csv.zip  
  inflating: bosch_data/test_date.csv.zip  
  inflating: bosch_data/test_numeric.csv.zip  
  inflating: bosch_data/train_categorical.csv.zip  
  inflating: bosch_data/train_date.csv.zip  
  inflating: bosch_data/train_numeric.csv.zip  


In [8]:
date = pd.read_csv('bosch_data/train_date.csv.zip', nrows=10000)
numeric = pd.read_csv('bosch_data/train_numeric.csv.zip', nrows=10000)
category = pd.read_csv('bosch_data/train_categorical.csv.zip', nrows=10000)

print(date.shape, numeric.shape, category.shape)

(10000, 1157) (10000, 970) (10000, 2141)


In [39]:
# Load numeric data
numeric = pd.read_csv('bosch_data/train_numeric.csv.zip', nrows=10000)

# 1. Basic shape and structure
print("🔹 Shape:", numeric.shape)
print("\n🔹 Preview of data:")
display(numeric.head())

print("\n🔹 Data types and non-null counts:")
numeric.info()

print("\n🔹 Basic statistics:")
display(numeric.describe())

🔹 Shape: (10000, 970)

🔹 Preview of data:


,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S2_F32,L0_S2_F36,L0_S2_F40,L0_S2_F44,L0_S2_F48,L0_S2_F52,L0_S2_F56,L0_S2_F60,L0_S2_F64,L0_S3_F68,L0_S3_F72,L0_S3_F76,L0_S3_F80,L0_S3_F84,L0_S3_F88,L0_S3_F92,L0_S3_F96,L0_S3_F100,L0_S4_F104,L0_S4_F109,L0_S5_F114,L0_S5_F116,L0_S6_F118,L0_S6_F122,L0_S6_F132,L0_S7_F136,L0_S7_F138,L0_S7_F142,L0_S8_F144,L0_S8_F146,L0_S8_F149,L0_S9_F155,L0_S9_F160,L0_S9_F165,L0_S9_F170,L0_S9_F175,L0_S9_F180,L0_S9_F185,L0_S9_F190,L0_S9_F195,L0_S9_F200,L0_S9_F205,L0_S9_F210,L0_S10_F219,L0_S10_F224,L0_S10_F229,L0_S10_F234,L0_S10_F239,L0_S10_F244,L0_S10_F249,L0_S10_F254,L0_S10_F259,L0_S10_F264,L0_S10_F269,L0_S10_F274,L0_S11_F282,L0_S11_F286,L0_S11_F290,L0_S11_F294,L0_S11_F298,L0_S11_F302,L0_S11_F306,L0_S11_F310,L0_S11_F314,L0_S11_F318,L0_S11_F322,L0_S11_F326,L0_S12_F330,L0_S12_F332,L0_S12_F334,L0_S12_F336,L0_S12_F338,L0_S12_F340,L0_S12_F342,L0_S12_F344,L0_S12_F346,L0_S12_F348,L0_S12_F350,L0_S12_F352,L0_S13_F354,L0_S13_F356,L0_S14_F358,L0_S14_F362,L0_S14_F366,L0_S14_F370,L0_S14_F374,L0_S14_F378,L0_S14_F382,L0_S14_F386,L0_S14_F390,L0_S15_F394,L0_S15_F397,L0_S15_F400,L0_S15_F403,L0_S15_F406,L0_S15_F409,L0_S15_F412,L0_S15_F415,L0_S15_F418,L0_S16_F421,L0_S16_F426,L0_S17_F431,L0_S17_F433,L0_S18_F435,L0_S18_F439,L0_S18_F449,L0_S19_F453,L0_S19_F455,L0_S19_F459,L0_S20_F461,L0_S20_F463,L0_S20_F466,L0_S21_F472,L0_S21_F477,L0_S21_F482,L0_S21_F487,L0_S21_F492,L0_S21_F497,L0_S21_F502,L0_S21_F507,L0_S21_F512,L0_S21_F517,L0_S21_F522,L0_S21_F527,L0_S21_F532,L0_S21_F537,L0_S22_F546,L0_S22_F551,L0_S22_F556,L0_S22_F561,L0_S22_F566,L0_S22_F571,L0_S22_F576,L0_S22_F581,L0_S22_F586,L0_S22_F591,L0_S22_F596,L0_S22_F601,L0_S22_F606,L0_S22_F611,L0_S23_F619,L0_S23_F623,L0_S23_F627,L0_S23_F631,L0_S23_F635,L0_S23_F639,L0_S23_F643,L0_S23_F647,L0_S23_F651,L0_S23_F655,L0_S23_F659,L0_S23_F663,L0_S23_F667,L0_S23_F671,L1_S24_F679,L1_S24_F683,L1_S24_F687,L1_S24_F691,L1_S24_F700,L1_S24_F719,L1_S24_F728,L1_S24_F733,L1_S24_F746,L1_S24_F751,L1_S24_F756,L1_S24_F761,L1_S24_F766,L1_S24_F775,L1_S24_F780,L1_S24_F785,L1_S24_F790,L1_S24_F795,L1_S24_F800,L1_S24_F802,L1_S24_F806,L1_S24_F808,L1_S24_F810,L1_S24_F812,L1_S24_F814,L1_S24_F816,L1_S24_F829,L1_S24_F834,L1_S24_F839,L1_S24_F844,L1_S24_F857,L1_S24_F862,L1_S24_F867,L1_S24_F872,L1_S24_F877,L1_S24_F882,L1_S24_F887,L1_S24_F892,L1_S24_F897,L1_S24_F902,L1_S24_F907,L1_S24_F920,L1_S24_F925,L1_S24_F930,L1_S24_F935,L1_S24_F948,L1_S24_F953,L1_S24_F958,L1_S24_F963,L1_S24_F968,L1_S24_F973,L1_S24_F978,L1_S24_F983,L1_S24_F988,L1_S24_F993,L1_S24_F998,L1_S24_F1000,L1_S24_F1002,L1_S24_F1004,L1_S24_F1006,L1_S24_F1008,L1_S24_F1010,L1_S24_F1012,L1_S24_F1014,L1_S24_F1016,L1_S24_F1021,L1_S24_F1026,L1_S24_F1031,L1_S24_F1036,L1_S24_F1041,L1_S24_F1046,L1_S24_F1051,L1_S24_F1056,L1_S24_F1068,L1_S24_F1072,L1_S24_F1079,L1_S24_F1083,L1_S24_F1087,L1_S24_F1094,L1_S24_F1098,L1_S24_F1102,L1_S24_F1106,L1_S24_F1110,L1_S24_F1118,L1_S24_F1122,L1_S24_F1126,L1_S24_F1130,L1_S24_F1134,L1_S24_F1145,L1_S24_F1148,L1_S24_F1161,L1_S24_F1166,L1_S24_F1170,L1_S24_F1172,L1_S24_F1174,L1_S24_F1176,L1_S24_F1180,L1_S24_F1184,L1_S24_F1197,L1_S24_F1202,L1_S24_F1207,L1_S24_F1212,L1_S24_F1225,L1_S24_F1230,L1_S24_F1235,L1_S24_F1240,L1_S24_F1245,L1_S24_F1250,L1_S24_F1255,L1_S24_F1260,L1_S24_F1265,L1_S24_F1270,L1_S24_F1275,L1_S24_F1288,L1_S24_F1293,L1_S24_F1298,L1_S24_F1303,L1_S24_F1316,L1_S24_F1321,L1_S24_F1326,L1_S24_F1331,L1_S24_F1336,L1_S24_F1341,L1_S24_F1346,L1_S24_F1351,L1_S24_F1356,L1_S24_F1361,L1_S24_F1366,L1_S24_F1371,L1_S24_F1376,L1_S24_F1381,L1_S24_F1386,L1_S24_F1391,L1_S24_F1396,L1_S24_F1401,L1_S24_F1406,L1_S24_F1411,L1_S24_F1416,L1_S24_F1421,L1_S24_F1426,L1_S24_F1431,L1_S24_F1436,L1_S24_F1441,L1_S24_F1446,L1_S24_F1451,L1_S24_F1463,L1_S24_F1467,L1_S24_F1474,L1_S24_F1478,L1_S24_F1482,L1_S24_F1486,L1_S24_F1490,L1_S24_F1494,L1_S24_F1498,L1_S24_F1502,L1_S24_F1506,L1_S24_F1512,L1_S24_F1514,L1_S24_F1516,L1_S24_F1518,L1_S24_F1520,L1_S24_F1539,L1_S24_F1544,L1_S24_F15


🔹 Data types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 970 entries, Id to Response
dtypes: float64(968), int64(2)
memory usage: 74.0 MB

🔹 Basic statistics:


Id     L0_S0_F0     L0_S0_F2     L0_S0_F4     L0_S0_F6  \
count  10000.000000  5733.000000  5733.000000  5733.000000  5733.000000   
mean    9959.598500    -0.001898    -0.002599     0.001011     0.000744   
std     5722.930873     0.079458     0.091942     0.213640     0.213748   
min        4.000000    -0.310000    -0.399000    -0.397000    -0.397000   
25%     5035.500000    -0.055000    -0.064000    -0.179000    -0.179000   
50%     9974.500000     0.003000     0.004000    -0.033000    -0.034000   
75%    14896.250000     0.056000     0.063000     0.294000     0.294000   
max    19923.000000     0.278000     0.280000     0.567000     0.566000   

          L0_S0_F8    L0_S0_F10    L0_S0_F12    L0_S0_F14    L0_S0_F16  \
count  5733.000000  5733.000000  5733.000000  5733.000000  5733.000000   
mean     -0.001164     0.004127     0.000347     0.002321    -0.000786   
std       0.094814     0.164772     0.019482     0.104789     0.115022   
min      -0.404000    -0.566000    -0.044000    -0.232000    -0.393000   
25%      -0.056000    -0.066000    -0.015000    -0.072000    -0.082000   
50%       0.031000     0.070000     0.000000    -0.032000     0.000000   
75%       0.074000     0.116000     0.015000     0.088000     0.076000   
max       0.292000     0.206000     0.089000     0.488000     0.393000   

         L0_S0_F18    L0_S0_F20    L0_S0_F22    L0_S1_F24    L0_S1_F28  \
count  5733.000000  5733.000000  5733.000000  5733.000000  5733.000000   
mean      0.000256     0.004859     0.004873     0.001445     0.000558   
std       0.111282     0.202510     0.202491     0.102356     0.108581   
min      -0.340000    -0.341000    -0.341000    -0.287000    -0.253000   
25%      -0.078000    -0.167000    -0.167000    -0.066000    -0.074000   
50%       0.002000     0.000000     0.000000    -0.004000    -0.012000   
75%       0.077000     0.168000     0.168000     0.080000     0.065000   
max       0.409000     0.586000     0.585000     0.288000     0.494000   

         L0_S2_F32    L0_S2_F36    L0_S2_F40    L0_S2_F44    L0_S2_F48  \
count  2849.000000  2849.000000  2849.000000  2849.000000  2849.000000   
mean     -0.001698     0.009906    -0.001653     0.001363     0.000026   
std       0.053317     0.204418     0.070224     0.263727     0.006455   
min      -0.213000    -0.308000    -0.192000    -0.373000    -0.037000   
25%      -0.013000    -0.128000    -0.092000    -0.215000    -0.002000   
50%      -0.013000    -0.054000     0.008000    -0.112000     0.001000   
75%      -0.013000     0.069000     0.008000     0.183000     0.004000   
max       0.787000     0.691000     0.808000     0.626000     0.011000   

       L0_S2_F52    L0_S2_F56    L0_S2_F60    L0_S2_F64    L0_S3_F68  \
count     2849.0  2849.000000  2849.000000  2849.000000  2888.000000   
mean         0.0     0.001014    -0.009059     0.002812    -0.000799   
std          0.0     0.059285     0.261163     0.060281     0.046835   
min          0.0    -0.190000    -0.365000    -0.176000    -0.191000   
25%          0.0     0.010000    -0.216000    -0.057000     0.009000   
50%          0.0     0.010000    -0.121000     0.001000     0.009000   
75%          0.0     0.010000     0.159000     0.056000     0.009000   
max          0.0     0.810000     0.627000     0.211000     0.109000   

         L0_S3_F72    L0_S3_F76    L0_S3_F80    L0_S3_F84  L0_S3_F88  \
count  2888.000000  2888.000000  2888.000000  2888.000000     2888.0   
mean      0.007664     0.000108     0.004670     0.000090        0.0   
std       0.256686     0.053283     0.269066     0.006537        0.0   
min      -0.274000    -0.151000    -0.350000    -0.038000        0.0   
25%      -0.201000    -0.051000    -0.252250    -0.002000        0.0   
50%      -0.095000    -0.051000    -0.022000     0.001000        0.0   
75%       0.189000     0.049000     0.233250     0.004000        0.0   
max       0.626000     0.149000     0.549000     0.011000        0.0   

         L0_S3_F92    L0_S3_F96   L0_S3_F1

## Remove Uninformative Columns from Numeric Features
1. Drop columns with more than 90% NaN

 Drop columns with more than 90% zeros (excluding 'Response' and 'Id')

 Drop columns with only one unique value


In [40]:
import plotly.graph_objects as go

# محاسبه نسبت NaN برای هر ستون
nan_ratio = numeric.isnull().mean()

# ساخت لیست آستانه‌ها
thresholds = [i / 100 for i in range(0, 101, 5)]  # [0.0, 0.05, ..., 1.0]
columns_remaining = [sum(nan_ratio <= t) for t in thresholds]

# رسم نمودار
fig = go.Figure()
fig.add_trace(go.Scatter(x=thresholds, y=columns_remaining, mode='lines+markers',
                         name='Remaining Columns'))

fig.update_layout(
    title='Effect of NaN Threshold on Remaining Columns',
    xaxis_title='NaN Threshold',
    yaxis_title='Number of Columns Remaining',
    template='plotly_white'
)

fig.show()


In [41]:

# 1. Drop columns with more than 90% NaN
nan_ratio = numeric.isnull().mean()
high_null_cols = nan_ratio[nan_ratio > 0.5].index
print(f" Dropping {len(high_null_cols)} columns with >85% NaN")
numeric_cleaned = numeric.drop(columns=high_null_cols)

# 2. Drop columns with more than 90% zeros (excluding 'Response' and 'Id')
zero_ratio = (numeric_cleaned.drop(['Id', 'Response'], axis=1) == 0).mean()
high_zero_cols = zero_ratio[zero_ratio > 0.90].index
print(f" Dropping {len(high_zero_cols)} columns with >90% zeros")
numeric_cleaned = numeric_cleaned.drop(columns=high_zero_cols)

# 3. Drop columns with only one unique value
nunique = numeric_cleaned.nunique()
constant_cols = nunique[nunique == 1].index
print(f" Dropping {len(constant_cols)} columns with only one unique value")
numeric_cleaned = numeric_cleaned.drop(columns=constant_cols)

# Final result
print(f"\n Final shape: {numeric_cleaned.shape}")
display(numeric_cleaned.head())


 Dropping 812 columns with >85% NaN
 Dropping 19 columns with >90% zeros
 Dropping 10 columns with only one unique value

 Final shape: (10000, 129)


,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S8_F144,L0_S8_F146,L0_S8_F149,L3_S29_F3315,L3_S29_F3318,L3_S29_F3321,L3_S29_F3324,L3_S29_F3327,L3_S29_F3330,L3_S29_F3333,L3_S29_F3336,L3_S29_F3339,L3_S29_F3342,L3_S29_F3345,L3_S29_F3348,L3_S29_F3351,L3_S29_F3354,L3_S29_F3357,L3_S29_F3360,L3_S29_F3367,L3_S29_F3370,L3_S29_F3373,L3_S29_F3376,L3_S29_F3379,L3_S29_F3382,L3_S29_F3385,L3_S29_F3388,L3_S29_F3395,L3_S29_F3398,L3_S29_F3401,L3_S29_F3404,L3_S29_F3407,L3_S29_F3412,L3_S29_F3421,L3_S29_F3424,L3_S29_F3427,L3_S29_F3430,L3_S29_F3433,L3_S29_F3436,L3_S29_F3439,L3_S29_F3442,L3_S29_F3449,L3_S29_F3452,L3_S29_F3455,L3_S29_F3458,L3_S29_F3461,L3_S29_F3464,L3_S29_F3467,L3_S29_F3470,L3_S29_F3473,L3_S29_F3476,L3_S29_F3479,L3_S29_F3482,L3_S30_F3494,L3_S30_F3499,L3_S30_F3504,L3_S30_F3509,L3_S30_F3514,L3_S30_F3519,L3_S30_F3524,L3_S30_F3529,L3_S30_F3534,L3_S30_F3539,L3_S30_F3544,L3_S30_F3549,L3_S30_F3554,L3_S30_F3559,L3_S30_F3564,L3_S30_F3569,L3_S30_F3574,L3_S30_F3579,L3_S30_F3584,L3_S30_F3589,L3_S30_F3604,L3_S30_F3609,L3_S30_F3624,L3_S30_F3629,L3_S30_F3634,L3_S30_F3639,L3_S30_F3644,L3_S30_F3649,L3_S30_F3664,L3_S30_F3669,L3_S30_F3674,L3_S30_F3679,L3_S30_F3684,L3_S30_F3689,L3_S30_F3704,L3_S30_F3709,L3_S30_F3744,L3_S30_F3749,L3_S30_F3754,L3_S30_F3759,L3_S30_F3764,L3_S30_F3769,L3_S30_F3774,L3_S30_F3779,L3_S30_F3784,L3_S30_F3789,L3_S30_F3794,L3_S30_F3799,L3_S30_F3804,L3_S30_F3809,L3_S30_F3814,L3_S30_F3819,L3_S30_F3824,L3_S30_F3829,L3_S33_F3855,L3_S33_F3857,L3_S33_F3859,L3_S33_F3861,L3_S33_F3863,L3_S33_F3865,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,0.083,-0.273,-0.273,-0.271,0.167,-0.157,0.0,0.001,0.030,-0.047,-0.119,0.040,-0.067,-0.058,0.044,0.044,0.010,0.041,-0.072,-0.024,-0.026,-0.257,0.250,0.218,0.040,0.044,0.090,0.036,-0.117,0.054,-0.119,0.044,-0.024,0.218,-0.072,0.250,-0.702,-0.702,-0.047,-0.058,0.065,-0.069,-0.027,-0.058,0.030,-0.067,0.041,-0.257,0.010,-0.026,-0.240,0.52,-0.240,0.52,0.015,0.015,0.067,0.0,0.003,0.003,-0.003,0.000,-0.137,0.086,-0.006,-0.003,0.159,-0.137,-0.020,-0.006,-0.010,0.159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.177,-0.160,0.091,0.275,0.010,-0.007,0.038,0.013,0.019,0.013,-0.009,0.012,-0.010,-0.060,0.013,-0.027,0.013,0.005,-0.119,-0.102,-0.048,-0.119,-0.119,0.026,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.084,-0.030,-0.127,-0.174,0.145,0.152,-0.006,-0.059,-0.064,-0.031,0.023,-0.010,0.113,0.238,-0.050,0.218,-0.174,-0.059,0.085,-0.015,-0.244,-0.050,-0.127,-0.006,-0.010,0.218,0.023,-0.050,0.082,0.082,-0.030,0.152,0.033,0.146,-0.022,0.152,-0.084,0.145,-0.031,0.238,-0.064,0.113,0.017,-0.48,0.017,-0.48,-0.019,-0.020,-0.071,0.0,-0.008,-0.008,-0.003,0.002,0.025,-0.046,-0.004,-0.003,0.039,0.025,0.030,-0.004,0.178,0.039,-0.003,0.016,0.033,0.255,-0.003,0.055,-0.002,0.060,-0.002,0.068,-0.008,0.049,-0.009,0.049,0.022,0.173,0.021,0.173,0.030,0.183,0.201,0.008,-0.055,-0.143,-0.079,-0.004,0.012,-0.024,-0.036,0.013,-0.017,0.013,0.011,0.014,0.001,-0.041,0.013,-0.019,0.013,-0.049,-0.029,-0.026,-0.210,-0.029,-0.029,-0.047,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,-0.147,0.250,0.250,0.057,-0.079,-0.157,0.0,0.001,0.049,-0.030,-0.099,0.070,0.014,0.004,0.006,0.064,0.040,-0.005,-0.129,-0.138,0.101,0.238,-0.017,-0.115,0.070,0.064,-0.223,0.023,0.223,0.047,-0.099,0.006,-0.138,-0.115,-0.129,-0.017,0.104,0.104,-0.030,0.004,0.071,0.014,-0.022,0.004,0.049,0.014,-0.005,0.238,0.040,0.101,0.017,-0.48,0.017,-0.48,-0.019,-0.009,-0.027,0.0,-0.009,-0.009,0.009,-0.003,0.025,0.007,0.001,0.009,-0.221,0.025,-0.013,0.001,-0.047,-0.221,0.009,0.016,-0.147,-0.078,0.001,-0.205,-0.016,-0.056,0.004,-0.010,-0.002,-0.041,-0.003,-0.041,0.012,-0.042,0.011,-0.042,-0.013,-0.042,0.054,0.007,-0.155,-0.044,0.059,0.063,0.006,-0.026,0.119,0.013,0.055,0.013,0.081,0.017,0.134,0.115,0.013,0.051,0.013,-0.069,0.199,0.191,0.236,0.199,0.199,0.108,0
3,9,-0.036,-0.0

In [42]:
numeric_nan_ratio = numeric_cleaned.isnull().mean().sort_values(ascending=False)
print(numeric_nan_ratio[numeric_nan_ratio > 0])  # فقط ستون‌هایی که NaN دارن


L0_S8_F146      0.4270
L0_S8_F149      0.4270
L0_S8_F144      0.4270
L0_S1_F24       0.4267
L0_S0_F0        0.4267
                 ...  
L3_S30_F3819    0.0534
L3_S30_F3814    0.0534
L3_S30_F3809    0.0534
L3_S30_F3829    0.0534
L3_S30_F3799    0.0534
Length: 127, dtype: float64


##  Smart NaN Filling Based on Column Characteristics



In [43]:
for col in numeric_cleaned.columns:
    missing_ratio = numeric_cleaned[col].isnull().mean()

    if missing_ratio > 0:
        # If the column has very few unique values (e.g. binary 0/1),
        # it's likely a flag or status column → fill with 0
        if numeric_cleaned[col].nunique() < 5:
            numeric_cleaned[col].fillna(0, inplace=True)

        # If missing ratio is low, mean imputation is safe
        elif missing_ratio < 0.2:
            numeric_cleaned[col].fillna(numeric_cleaned[col].mean(), inplace=True)

        # Otherwise, use median to avoid the influence of outliers
        else:
            numeric_cleaned[col].fillna(numeric_cleaned[col].median(), inplace=True)


In [44]:

print("Total remaining NaNs:", numeric_cleaned.isnull().sum().sum())


Total remaining NaNs: 0


## categorical

In [34]:
# Load categorical data (10,000 rows for demo)
categorical = pd.read_csv('bosch_data/train_categorical.csv.zip', nrows=10000)

# 1. Basic info
print("Shape:", categorical.shape)
print("\nPreview:")
display(categorical.head())

print("\nData types and nulls:")
categorical.info()

Shape: (10000, 2141)

Preview:


Id L0_S1_F25 L0_S1_F27 L0_S1_F29 L0_S1_F31 L0_S2_F33 L0_S2_F35 L0_S2_F37  \
0   4       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1   6       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2   7       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
3   9       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
4  11       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

  L0_S2_F39 L0_S2_F41 L0_S2_F43 L0_S2_F45 L0_S2_F47 L0_S2_F49 L0_S2_F51  \
0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
3       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
4       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

  L0_S2_F53 L0_S2_F55 L0_S2_F57 L0_S2_F59 L0_S2_F61 L0_S2_F63 L0_S2_F65  \
0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
3       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
4       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

  L0_S2_F67  L0_S3_F69  L0_S3_F71  L0_S3_F73  L0_S3_F75  L0_S3_F77  L0_S3_F79  \
0       NaN        NaN        NaN        NaN        NaN        NaN        NaN   
1       NaN        NaN        NaN        NaN        NaN        NaN        NaN   
2       NaN        NaN        NaN        NaN        NaN        NaN        NaN   
3       NaN        NaN        NaN        NaN        NaN        NaN        NaN   
4       NaN        NaN        NaN        NaN        NaN        NaN        NaN   

   L0_S3_F81  L0_S3_F83  L0_S3_F85  L0_S3_F87  L0_S3_F89  L0_S3_F91  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN        NaN        NaN        NaN        NaN   
2        NaN        NaN        NaN        NaN        NaN        NaN   
3        NaN        NaN        NaN        NaN        NaN        NaN   
4        NaN        NaN        NaN        NaN        NaN        NaN   

   L0_S3_F93  L0_S3_F95  L0_S3_F97  L0_S3_F99  L0_S3_F101  L0_S3_F103  \
0        NaN        NaN        NaN        NaN         NaN         NaN   
1        NaN        NaN        NaN        NaN         NaN         NaN   
2        NaN        NaN        NaN        NaN         NaN         NaN   
3        NaN        NaN        NaN        NaN         NaN         NaN   
4        NaN        NaN        NaN        NaN         NaN         NaN   

  L0_S4_F105 L0_S4_F107 L0_S4_F108  L0_S4_F110  L0_S4_F112  L0_S4_F113  \
0        NaN        NaN        NaN         NaN         NaN         NaN   
1        NaN        NaN        NaN         NaN         NaN         NaN   
2        NaN        NaN        NaN         NaN         NaN         NaN   
3        NaN        NaN        NaN         NaN         NaN         NaN   
4        NaN        NaN        NaN         NaN         NaN         NaN   

   L0_S6_F119  L0_S6_F121  L0_S6_F123  L0_S6_F125  L0_S6_F126  L0_S6_F128  \
0         NaN         NaN         NaN         NaN         NaN         NaN   
1         NaN         NaN         NaN         NaN         NaN         NaN   
2         NaN         NaN         NaN         NaN         NaN         NaN   
3         NaN         NaN         NaN         NaN         NaN         NaN   
4         NaN         NaN         NaN         NaN         NaN         NaN   

   L0_S6_F129  L0_S6_F131  L0_S6_F133  L0_S6_F135 L0_S9_F151  L0_S9_F153  \
0         NaN         NaN         NaN         NaN        NaN         NaN   
1         NaN         NaN         NaN         NaN        NaN         NaN   
2         NaN         NaN         NaN         NaN        NaN         NaN   
3         NaN         NaN         NaN         NaN        NaN 


Data types and nulls:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 2141 entries, Id to L3_S49_F4240
dtypes: float64(1375), int64(1), object(765)
memory usage: 163.3+ MB


In [46]:


# Calculate NaN ratio for each column
cat_nan_ratio = categorical.isnull().mean().sort_values(ascending=False)


thresholds = [i / 100 for i in range(0, 101, 5)]
remaining_columns = [sum(cat_nan_ratio <= t) for t in thresholds]

fig = go.Figure()
fig.add_trace(go.Scatter(x=thresholds, y=remaining_columns, mode='lines+markers',
                         name='Remaining Categorical Columns'))

fig.update_layout(
    title='Effect of NaN Threshold on Categorical Columns',
    xaxis_title='NaN Threshold',
    yaxis_title='Number of Columns Remaining',
    template='plotly_white'
)

fig.show()



## Remove Uninformative Columns from Categorical Features

In [47]:

cat_nan_ratio = categorical.isnull().mean()
cat_cols_to_keep = cat_nan_ratio[cat_nan_ratio <= 0.5].index
categorical_cleaned = categorical[cat_cols_to_keep]

print(f"Final shape of categorical_cleaned: {categorical_cleaned.shape}")

print(f"Final shape after cleaning: {categorical_cleaned.shape}")


Final shape of categorical_cleaned: (10000, 64)
Final shape after cleaning: (10000, 64)


In [48]:
categorical_cleaned.to_csv('categorical_cleaned.csv', index=False)

In [49]:
categorical_cleaned.isnull().mean().sort_values(ascending=False)


,0
L3_S29_F3475,0.3690
L3_S29_F3478,0.3690
L3_S29_F3481,0.3690
L3_S29_F3317,0.3394
L3_S29_F3332,0.3394
...,...
L3_S29_F3472,0.3394
L3_S29_F3490,0.3394
L3_S29_F3487,0.3394
L3_S29_F3493,0.3394


In [50]:
# 1. Drop columns with only one unique non-null value
unique_vals = categorical_cleaned.nunique(dropna=True)
cat_cols_final = unique_vals[unique_vals > 1].index
categorical_cleaned = categorical_cleaned[cat_cols_final]

# 2. Fill remaining NaNs with 'missing'
categorical_cleaned = categorical_cleaned.fillna("missing")

# 3. Final shape info
print(f"Final shape after dropping low-info columns: {categorical_cleaned.shape}")
print(f"Example columns: {categorical_cleaned.columns[:5].tolist()}")
Final shape after dropping low-info columns: (10000, 1)
Example columns: ['Id']

Final shape after dropping low-info columns: (10000, 1)
Example columns: ['Id']


## Why I Dropped the Categorical Features
Most categorical columns had:

Over 50% missing values

Only one unique value (e.g., "T1")

After cleaning, only the Id column remained. So, categorical data had no useful information for modeling and was dropped to simplify the process.



## Dates

In [52]:
# Load date data (first 10,000 rows for exploration)
date = pd.read_csv('bosch_data/train_date.csv.zip', nrows=10000)

# 1. Overview
print("Shape:", date.shape)
print("\nSample:")
display(date.head())

print("\nInfo:")
date.info()

Shape: (10000, 1157)

Sample:


,Id,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,L0_S0_D19,L0_S0_D21,L0_S0_D23,L0_S1_D26,L0_S1_D30,L0_S2_D34,L0_S2_D38,L0_S2_D42,L0_S2_D46,L0_S2_D50,L0_S2_D54,L0_S2_D58,L0_S2_D62,L0_S2_D66,L0_S3_D70,L0_S3_D74,L0_S3_D78,L0_S3_D82,L0_S3_D86,L0_S3_D90,L0_S3_D94,L0_S3_D98,L0_S3_D102,L0_S4_D106,L0_S4_D111,L0_S5_D115,L0_S5_D117,L0_S6_D120,L0_S6_D124,L0_S6_D127,L0_S6_D130,L0_S6_D134,L0_S7_D137,L0_S7_D139,L0_S7_D140,L0_S7_D141,L0_S7_D143,L0_S8_D145,L0_S8_D147,L0_S8_D148,L0_S8_D150,L0_S9_D152,L0_S9_D157,L0_S9_D162,L0_S9_D167,L0_S9_D172,L0_S9_D177,L0_S9_D182,L0_S9_D187,L0_S9_D192,L0_S9_D197,L0_S9_D202,L0_S9_D207,L0_S9_D212,L0_S10_D216,L0_S10_D221,L0_S10_D226,L0_S10_D231,L0_S10_D236,L0_S10_D241,L0_S10_D246,L0_S10_D251,L0_S10_D256,L0_S10_D261,L0_S10_D266,L0_S10_D271,L0_S10_D276,L0_S11_D280,L0_S11_D284,L0_S11_D288,L0_S11_D292,L0_S11_D296,L0_S11_D300,L0_S11_D304,L0_S11_D308,L0_S11_D312,L0_S11_D316,L0_S11_D320,L0_S11_D324,L0_S11_D328,L0_S12_D331,L0_S12_D333,L0_S12_D335,L0_S12_D337,L0_S12_D339,L0_S12_D341,L0_S12_D343,L0_S12_D345,L0_S12_D347,L0_S12_D349,L0_S12_D351,L0_S12_D353,L0_S13_D355,L0_S13_D357,L0_S14_D360,L0_S14_D364,L0_S14_D368,L0_S14_D372,L0_S14_D376,L0_S14_D380,L0_S14_D384,L0_S14_D388,L0_S14_D392,L0_S15_D395,L0_S15_D398,L0_S15_D401,L0_S15_D404,L0_S15_D407,L0_S15_D410,L0_S15_D413,L0_S15_D416,L0_S15_D419,L0_S16_D423,L0_S16_D428,L0_S17_D432,L0_S17_D434,L0_S18_D437,L0_S18_D441,L0_S18_D444,L0_S18_D447,L0_S18_D451,L0_S19_D454,L0_S19_D456,L0_S19_D457,L0_S19_D458,L0_S19_D460,L0_S20_D462,L0_S20_D464,L0_S20_D465,L0_S20_D467,L0_S21_D469,L0_S21_D474,L0_S21_D479,L0_S21_D484,L0_S21_D489,L0_S21_D494,L0_S21_D499,L0_S21_D504,L0_S21_D509,L0_S21_D514,L0_S21_D519,L0_S21_D524,L0_S21_D529,L0_S21_D534,L0_S21_D539,L0_S22_D543,L0_S22_D548,L0_S22_D553,L0_S22_D558,L0_S22_D563,L0_S22_D568,L0_S22_D573,L0_S22_D578,L0_S22_D583,L0_S22_D588,L0_S22_D593,L0_S22_D598,L0_S22_D603,L0_S22_D608,L0_S22_D613,L0_S23_D617,L0_S23_D621,L0_S23_D625,L0_S23_D629,L0_S23_D633,L0_S23_D637,L0_S23_D641,L0_S23_D645,L0_S23_D649,L0_S23_D653,L0_S23_D657,L0_S23_D661,L0_S23_D665,L0_S23_D669,L0_S23_D673,L1_S24_D677,L1_S24_D681,L1_S24_D685,L1_S24_D689,L1_S24_D693,L1_S24_D697,L1_S24_D702,L1_S24_D707,L1_S24_D712,L1_S24_D716,L1_S24_D721,L1_S24_D725,L1_S24_D730,L1_S24_D735,L1_S24_D739,L1_S24_D743,L1_S24_D748,L1_S24_D753,L1_S24_D758,L1_S24_D763,L1_S24_D768,L1_S24_D772,L1_S24_D777,L1_S24_D782,L1_S24_D787,L1_S24_D792,L1_S24_D797,L1_S24_D801,L1_S24_D804,L1_S24_D807,L1_S24_D809,L1_S24_D811,L1_S24_D813,L1_S24_D815,L1_S24_D818,L1_S24_D822,L1_S24_D826,L1_S24_D831,L1_S24_D836,L1_S24_D841,L1_S24_D846,L1_S24_D850,L1_S24_D854,L1_S24_D859,L1_S24_D864,L1_S24_D869,L1_S24_D874,L1_S24_D879,L1_S24_D884,L1_S24_D889,L1_S24_D894,L1_S24_D899,L1_S24_D904,L1_S24_D909,L1_S24_D913,L1_S24_D917,L1_S24_D922,L1_S24_D927,L1_S24_D932,L1_S24_D937,L1_S24_D941,L1_S24_D945,L1_S24_D950,L1_S24_D955,L1_S24_D960,L1_S24_D965,L1_S24_D970,L1_S24_D975,L1_S24_D980,L1_S24_D985,L1_S24_D990,L1_S24_D995,L1_S24_D999,L1_S24_D1001,L1_S24_D1003,L1_S24_D1005,L1_S24_D1007,L1_S24_D1009,L1_S24_D1011,L1_S24_D1013,L1_S24_D1015,L1_S24_D1018,L1_S24_D1023,L1_S24_D1028,L1_S24_D1033,L1_S24_D1038,L1_S24_D1043,L1_S24_D1048,L1_S24_D1053,L1_S24_D1058,L1_S24_D1062,L1_S24_D1066,L1_S24_D1070,L1_S24_D1074,L1_S24_D1077,L1_S24_D1081,L1_S24_D1085,L1_S24_D1089,L1_S24_D1092,L1_S24_D1096,L1_S24_D1100,L1_S24_D1104,L1_S24_D1108,L1_S24_D1112,L1_S24_D1116,L1_S24_D1120,L1_S24_D1124,L1_S24_D1128,L1_S24_D1132,L1_S24_D1135,L1_S24_D1138,L1_S24_D1141,L1_S24_D1143,L1_S24_D1146,L1_S24_D1149,L1_S24_D1151,L1_S24_D1153,L1_S24_D1155,L1_S24_D1158,L1_S24_D1163,L1_S24_D1168,L1_S24_D1171,L1_S24_D1173,L1_S24_D1175,L1_S24_D1178,L1_S24_D1182,L1_S24_D1186,L1_S24_D1190,L1_S24_D1194,L1_S24_D1199,L1_S24_D1204,L1_S24_D1209,L1_S24_D1214,L1_S24_D1218,L1_S24_D1222,L1_S24_D1227,L1_S24_D1232,L1_S24_D1237,L1_S24_D1242,L1_S24_D1247,L1_S24_D1252,L1_S24_D1257,L1_S24_D1262,L1_S24_D1267,L1_S24_D1272,L1_S24_D1277,L1_S24_D1281,L1_S24_D1285,L1_S24_D1290,L1_S24_D1295,L1_S24_D13


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 1157 entries, Id to L3_S51_D4263
dtypes: float64(1156), int64(1)
memory usage: 88.3 MB


In [54]:
# NaN ratio for date columns (excluding 'Id')
date_nan_ratio = date.drop(columns='Id').isnull().mean().sort_values(ascending=False)

# Thresholds and remaining columns
thresholds = [i / 100 for i in range(0, 101, 5)]
remaining_columns = [sum(date_nan_ratio <= t) for t in thresholds]

# Plot
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=thresholds, y=remaining_columns, mode='lines+markers',
                         name='Remaining Date Columns'))

fig.update_layout(
    title='Effect of NaN Threshold on Date Columns',
    xaxis_title='NaN Threshold',
    yaxis_title='Number of Columns Remaining',
    template='plotly_white'
)

fig.show()




In [57]:
#  Drop columns with more than 90% missing values
null_ratio = date.isnull().mean()
high_null_cols = null_ratio[null_ratio > 0.90].index
print(f"\nDropping {len(high_null_cols)} columns with >90% missing values")
date_cleaned = date.drop(columns=high_null_cols)

#  Drop columns with only one unique value
nunique = date_cleaned.nunique(dropna=True)
constant_cols = nunique[nunique == 1].index
print(f"Dropping {len(constant_cols)} columns with only one unique value")
date_cleaned = date_cleaned.drop(columns=constant_cols)


#  Final shape
print(f"Final shape after cleaning: {date_cleaned.shape}")


Dropping 762 columns with >90% missing values
Dropping 0 columns with only one unique value
Final shape after cleaning: (10000, 395)


Extract processing time features:
- min_time: timestamp when processing started
- max_time: timestamp when processing ended
- processing_time: total time the part spent in the production process
These features summarize the temporal behavior of each part in the line
and may help the model detect anomalies related to unusually long or short processing durations.

In [58]:
# Extract only time columns (exclude 'Id')
date_features = date_cleaned.drop(columns='Id')

# Compute min, max, and processing duration for each row
date_cleaned['min_time'] = date_features.min(axis=1)
date_cleaned['max_time'] = date_features.max(axis=1)
date_cleaned['processing_time'] = date_cleaned['max_time'] - date_cleaned['min_time']

# Keep only essential columns: Id + new time features
date_summary = date_cleaned[['Id', 'min_time', 'max_time', 'processing_time']]

# Preview result
print("Date summary shape:", date_summary.shape)
display(date_summary.head(10))


Date summary shape: (10000, 4)


,Id,min_time,max_time,processing_time
0,4,82.24,87.29,5.05
1,6,1313.12,1315.75,2.63
2,7,1618.70,1624.42,5.72
3,9,1149.20,1154.16,4.96
4,11,602.64,606.02,3.38
5,13,1331.66,1339.73,8.07
6,14,1662.63,1664.04,1.41
7,16,791.22,804.36,13.14
8,18,517.64,518.08,0.44
9,23,156.27,157.89,1.62


Note:
The time values in train_date.csv are anonymized numeric values.
Their absolute unit (e.g., minutes, seconds) is unknown, but their relative difference is meaningful.
Therefore, we can use min_time, max_time, and processing_time as useful features,
but we should avoid interpreting them in real-world time units.

## merge

In [59]:
# Merge numeric and date summary features on 'Id'
final_df = pd.merge(numeric_cleaned, date_summary, on='Id', how='left')

# Final shape and preview
print(f"\nFinal merged shape: {final_df.shape}")
display(final_df.head())



Final merged shape: (10000, 132)


,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,L0_S0_F20,L0_S0_F22,L0_S1_F24,L0_S1_F28,L0_S8_F144,L0_S8_F146,L0_S8_F149,L3_S29_F3315,L3_S29_F3318,L3_S29_F3321,L3_S29_F3324,L3_S29_F3327,L3_S29_F3330,L3_S29_F3333,L3_S29_F3336,L3_S29_F3339,L3_S29_F3342,L3_S29_F3345,L3_S29_F3348,L3_S29_F3351,L3_S29_F3354,L3_S29_F3357,L3_S29_F3360,L3_S29_F3367,L3_S29_F3370,L3_S29_F3373,L3_S29_F3376,L3_S29_F3379,L3_S29_F3382,L3_S29_F3385,L3_S29_F3388,L3_S29_F3395,L3_S29_F3398,L3_S29_F3401,L3_S29_F3404,L3_S29_F3407,L3_S29_F3412,L3_S29_F3421,L3_S29_F3424,L3_S29_F3427,L3_S29_F3430,L3_S29_F3433,L3_S29_F3436,L3_S29_F3439,L3_S29_F3442,L3_S29_F3449,L3_S29_F3452,L3_S29_F3455,L3_S29_F3458,L3_S29_F3461,L3_S29_F3464,L3_S29_F3467,L3_S29_F3470,L3_S29_F3473,L3_S29_F3476,L3_S29_F3479,L3_S29_F3482,L3_S30_F3494,L3_S30_F3499,L3_S30_F3504,L3_S30_F3509,L3_S30_F3514,L3_S30_F3519,L3_S30_F3524,L3_S30_F3529,L3_S30_F3534,L3_S30_F3539,L3_S30_F3544,L3_S30_F3549,L3_S30_F3554,L3_S30_F3559,L3_S30_F3564,L3_S30_F3569,L3_S30_F3574,L3_S30_F3579,L3_S30_F3584,L3_S30_F3589,L3_S30_F3604,L3_S30_F3609,L3_S30_F3624,L3_S30_F3629,L3_S30_F3634,L3_S30_F3639,L3_S30_F3644,L3_S30_F3649,L3_S30_F3664,L3_S30_F3669,L3_S30_F3674,L3_S30_F3679,L3_S30_F3684,L3_S30_F3689,L3_S30_F3704,L3_S30_F3709,L3_S30_F3744,L3_S30_F3749,L3_S30_F3754,L3_S30_F3759,L3_S30_F3764,L3_S30_F3769,L3_S30_F3774,L3_S30_F3779,L3_S30_F3784,L3_S30_F3789,L3_S30_F3794,L3_S30_F3799,L3_S30_F3804,L3_S30_F3809,L3_S30_F3814,L3_S30_F3819,L3_S30_F3824,L3_S30_F3829,L3_S33_F3855,L3_S33_F3857,L3_S33_F3859,L3_S33_F3861,L3_S33_F3863,L3_S33_F3865,Response,min_time,max_time,processing_time
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,0.083,-0.273,-0.273,-0.271,0.167,-0.157,0.0,0.001,0.030,-0.047,-0.119,0.040,-0.067,-0.058,0.044,0.044,0.010,0.041,-0.072,-0.024,-0.026,-0.257,0.250,0.218,0.040,0.044,0.090,0.036,-0.117,0.054,-0.119,0.044,-0.024,0.218,-0.072,0.250,-0.702,-0.702,-0.047,-0.058,0.065,-0.069,-0.027,-0.058,0.030,-0.067,0.041,-0.257,0.010,-0.026,-0.240,0.52,-0.240,0.52,0.015,0.015,0.067,0.0,0.003,0.003,-0.003,0.000,-0.137,0.086,-0.006,-0.003,0.159,-0.137,-0.020,-0.006,-0.010,0.159,0.000,-0.011,0.027,-0.018,0.001,-0.005,-0.006,-0.015,-0.002,-0.010,-0.002,-0.010,-0.002,-0.010,-0.004,-0.020,-0.005,-0.020,-0.002,-0.019,-0.003,-0.001,-0.177,-0.160,0.091,0.275,0.010,-0.007,0.038,0.013,0.019,0.013,-0.009,0.012,-0.010,-0.060,0.013,-0.027,0.013,0.005,-0.119,-0.102,-0.048,-0.119,-0.119,0.026,0,82.24,87.29,5.05
1,6,0.003,0.004,-0.033,-0.034,0.031,0.070,0.000,-0.032,0.000,0.002,0.000,0.000,-0.004,-0.012,0.000,0.0,0.001,-0.084,-0.030,-0.127,-0.174,0.145,0.152,-0.006,-0.059,-0.064,-0.031,0.023,-0.010,0.113,0.238,-0.050,0.218,-0.174,-0.059,0.085,-0.015,-0.244,-0.050,-0.127,-0.006,-0.010,0.218,0.023,-0.050,0.082,0.082,-0.030,0.152,0.033,0.146,-0.022,0.152,-0.084,0.145,-0.031,0.238,-0.064,0.113,0.017,-0.48,0.017,-0.48,-0.019,-0.020,-0.071,0.0,-0.008,-0.008,-0.003,0.002,0.025,-0.046,-0.004,-0.003,0.039,0.025,0.030,-0.004,0.178,0.039,-0.003,0.016,0.033,0.255,-0.003,0.055,-0.002,0.060,-0.002,0.068,-0.008,0.049,-0.009,0.049,0.022,0.173,0.021,0.173,0.030,0.183,0.201,0.008,-0.055,-0.143,-0.079,-0.004,0.012,-0.024,-0.036,0.013,-0.017,0.013,0.011,0.014,0.001,-0.041,0.013,-0.019,0.013,-0.049,-0.029,-0.026,-0.210,-0.029,-0.029,-0.047,0,1313.12,1315.75,2.63
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,-0.147,0.250,0.250,0.057,-0.079,-0.157,0.0,0.001,0.049,-0.030,-0.099,0.070,0.014,0.004,0.006,0.064,0.040,-0.005,-0.129,-0.138,0.101,0.238,-0.017,-0.115,0.070,0.064,-0.223,0.023,0.223,0.047,-0.099,0.006,-0.138,-0.115,-0.129,-0.017,0.104,0.104,-0.030,0.004,0.071,0.014,-0.022,0.004,0.049,0.014,-0.005,0.238,0.040,0.101,0.017,-0.48,0.017,-0.48,-0.019,-0.009,-0.027,0.0,-0.009,-0.009,0.009,-0.003,0.025,0.007,0.001,0.009,-0.221,0.025,-0.013,0.001,-0.047,-0.221,0.009,0.016,-0.147,-0.078,0.001,-0.205,-0.016,-0.056,0.004,-0.010,-0.002,-0.041,-0.003,-0.041,0.012,-0.042,0.011,-0.042,-0.013,-0.042,0.054

In [63]:
total_nans = final_df.isnull().sum().sum()
print(f"Total NaNs: {total_nans}")


Total NaNs: 36


In [67]:
nan_counts = final_df.isnull().sum()
print(nan_counts[nan_counts > 0])


min_time           12
max_time           12
processing_time    12
dtype: int64


In [68]:
# Drop rows with missing time features
final_df = final_df.dropna(subset=['min_time', 'max_time', 'processing_time'])

print("✅ After dropping rows with missing time info:")
print("Remaining NaNs:", final_df.isnull().sum().sum())
print("New shape:", final_df.shape)


✅ After dropping rows with missing time info:
Remaining NaNs: 0
New shape: (9988, 132)


In [69]:
non_numeric = final_df.select_dtypes(exclude=['number']).columns
print("Non-numeric columns:", list(non_numeric))


Non-numeric columns: []


In [70]:
print(final_df['Response'].value_counts(normalize=True))


Response
0    0.994694
1    0.005306
Name: proportion, dtype: float64


In [66]:
-final_df.describe().T[['min', 'max']].sort_values(by='max', ascending=False).head(10)


,min,max
Id,-4.000,-19923.000
max_time,-0.480,-1718.310
min_time,-0.410,-1716.900
processing_time,-0.000,-105.880
Response,-0.000,-1.000
L3_S29_F3433,0.091,-0.833
L3_S29_F3385,0.239,-0.761
L3_S29_F3321,0.239,-0.761
L3_S29_F3379,0.276,-0.724
L3_S29_F3461,0.354,-0.646


## Save dataset

In [71]:
final_df.to_csv('bosch_cleaned_for_eda.csv', index=False)
print("✅ Dataset saved as: bosch_cleaned_for_eda.csv")


✅ Dataset saved as: bosch_cleaned_for_eda.csv
